<a href="https://colab.research.google.com/github/bhadreshpsavani/60DaysofUdacityCodeChallenge/blob/master/EncryptedComputationInPySyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch as th
!pip install syft
import syft as sy
hook= sy.TorchHook(th)

     |████████████████████████████████| 225kB 4.8MB/s 
     |████████████████████████████████| 389kB 41.6MB/s 
     |████████████████████████████████| 460kB 39.2MB/s 
     |████████████████████████████████| 204kB 41.8MB/s 
     |████████████████████████████████| 256kB 42.1MB/s 
     |████████████████████████████████| 81kB 25.5MB/s 
     |████████████████████████████████| 2.1MB 42.8MB/s 
     |████████████████████████████████| 51kB 21.7MB/s 
     |████████████████████████████████| 276kB 51.7MB/s 
     |████████████████████████████████| 122kB 43.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
   

W0727 05:50:01.402967 140142395791232 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0727 05:50:01.430631 140142395791232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
from torch import nn, optim

In [0]:
bob=sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice=sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker=sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [0]:
x=th.tensor([1, 2, 4, -1])
y=th.tensor([-1, -3, -3, -2])

In [0]:
x=x.share(alice, bob, crypto_provider=secure_worker)
y=y.share(alice, bob, crypto_provider=secure_worker)

In [72]:
z=x+y
z.get()

tensor([ 0, -1,  1, -3])

In [73]:
z=x-y
z.get()

tensor([2, 5, 7, 1])

In [74]:
z=x*y
z.get()

tensor([ -1,  -6, -12,   2])

In [75]:
z=x>y
z.get()

tensor([1, 1, 1, 1])

In [76]:
z=x<y
z.get()

tensor([0, 0, 0, 0])

In [77]:
z=x==y
z.get()

tensor([0, 0, 0, 0])

In [0]:
x=th.tensor([1, 2, 4, -1])
y=th.tensor([-1, -3, -3, -2])

x=x.fix_prec().share(alice, bob, crypto_provider=secure_worker)
y=y.fix_prec().share(alice, bob, crypto_provider=secure_worker)

In [79]:
z=x+y
z.get().float_prec()

tensor([ 0., -1.,  1., -3.])

#Encrypted Database

In [0]:
import string

In [0]:
char2index={}
index2char={}

In [0]:
for i,char in enumerate(' '+string.ascii_lowercase+'0123456789'+string.punctuation):
  char2index[char]=i
  index2char[i]=char

In [0]:
str_input="hello"
max_len=8

In [0]:
def string2values(str_input, max_len=8):
  str_input=str_input[:max_len].lower()

  #pad string shorter than string input
  if(len(str_input) < max_len):
    str_input=str_input+'.'*(max_len-len(str_input))

  values=list()
  for char in str_input:
    values.append(char2index[char])
    
  return th.tensor(values).long()

def values2string(input_value):
  a=""
  for values in input_value:
    a+=index2char[int(values)]
  return a

def one_hot(index, length):
  vect=th.zeros(length).long()
  vect[index]=1
  return vect

def string2one_hot_matrix(str_input, max_len=8):
  str_input=str_input[:max_len]
  str_input=str_input.lower()

  #pad string shorter than string input
  if(len(str_input) < max_len):
    str_input=str_input+'.'*(max_len-len(str_input))
    
  char_vector=list()
  for char in str_input:
    char_v=one_hot(char2index[char], len(index2char)).unsqueeze(0)
    char_vector.append(char_v)
  return th.cat(char_vector, dim=0)

def str_compr(str1, str2):
  vect=(str1*str2).sum(1)
  x=vect[0]
  for i in range(vect.shape[0]-1):
    x= x * vect[i + 1] 
  return x

In [0]:
class EncryptedDB():
  
  def __init__(self, *owners,  max_key_len=8, max_value_len=8):
    self.max_key_len=max_key_len
    self.max_value_len=max_value_len
    self.owners=owners
    self.keys=list()
    self.values=list()
    
  def add_entry(self, key, value):
    key=string2one_hot_matrix(key)
    key=key.share(*self.owners)
    self.keys.append(key)
    value=string2values(value, max_len=self.max_value_len)
    value=value.share(*self.owners)
    self.values.append(value)
    
  def query(self, query_str):
    query_matrix=string2one_hot_matrix(query_str)
    query_matrix=query_matrix.share(*self.owners)
    key_matches=list()
    for key in self.keys:
      key_match=str_compr(key,query_matrix)
      key_matches.append(key_match)

    result= self.values[0] * key_matches[0]

    for i in range(len(self.values)-1):
      result+=(self.values[i+1] * key_matches[i+1])
     
    result=result.get()

    return values2string(result).replace(".","")

In [0]:
db=EncryptedDB(bob,alice,secure_worker, max_value_len=256)

In [0]:
db.add_entry("bob", "(123) 756-877")
db.add_entry("alice", "(129) 756-878")
db.add_entry("john", "(128) 750-877")
db.add_entry("key", "value are in json format")

In [98]:
db.query("bob")

'(123) 756-877'